In [26]:
import json

with open("./data/q_list_test.json", 'r', encoding='utf-8') as f:
    q_list_test = json.load(f)

with open("./data/QA_dict.json", 'r', encoding='utf-8') as f:
    QA_dict = json.load(f)

with open("./data/test_dict.json", 'r', encoding='utf-8') as f:
    test_dict = json.load(f)

with open("./data/QA_test_Qwen.json", 'r', encoding='utf-8') as f:
    QA_test_Qwen = json.load(f)

In [2]:
from utils_bm25 import creat_retriever

def test_QA(question):
    retriever = creat_retriever("QA")
    similarities = retriever.retrieve(queries=[question], k=1)
    mate_query = similarities[0][0]['text']
    #real_answear = test_dict[question]
    mat_answear = query_dict[mate_query]
    print("根据您输入的问题，请为您推荐以下QA内容：")
    print(f"Q:{mate_query}\n")
    print(f"Q:{mat_answear}\n")
    

def test_all(question):
    n = 0
    similarities = retriever.retrieve(queries=[question], k=1)
    mate_query = similarities[0][0]['text']
    real_answear = test_dict[question]
    mat_answear = query_dict[mate_query]
    if mat_answear == real_answear:
        return n+1
    else:
        return n

def test_pare(question):
    print(f"输入的问题是{question}")
    print("以下是QA配对：——————")
    test_QA(question)
    print("以下是合同条文搜索：——————")
    retriever = creat_retriever("Contract")
    similarities = retriever.retrieve(queries=[question], k=10)
    for i in range(10):
        print(similarities[0][i],"/n")
    print("以下是BGE合同条文搜索：——————")

def similarities_QA(question,k=1):
    retriever = creat_retriever("QA")
    similarities = retriever.retrieve(queries=[question], k=k)
    mate_query = similarities[0]
    return mate_query

In [47]:
from tqdm import tqdm
num = 0
total = len(QA_test_Qwen)
request_num = 0
TT=0
TF=0
FT=0
FF=0
err_num = 0
err_list=[]
for i in tqdm(range(total)):
    question = QA_test_Qwen[i][0]
    similarities = similarities_QA(question,k=1)
    q = similarities[0]['text']
    a = QA_dict[q]
    res = call_with_messages_qa(question,q,a)
    if res == "已解答":
        if QA_test_Qwen[i][1] == q:
            TT += 1
        else:
            TF += 1
            err_list.append(["TF",question,QA_test_Qwen[i][1],q,a,res])
    else:
        if QA_test_Qwen[i][1] == q:
            FT += 1
            err_list.append(["FT",question,QA_test_Qwen[i][1],q,a,res])
        else:
            FF += 1

acc = (TT+FF)/total
print(f"共计测试{total}条，正确数量为{num}，准确率为{acc}")
print(TT,TF)
print(FT,FF)

100%|██████████| 318/318 [03:19<00:00,  1.60it/s]

共计测试318条，正确数量为0，准确率为0.9559748427672956
284 4
10 20


In [55]:
for i in range(len(err_list)):
    print(f"【错误类型】：{err_list[i][0]},\n【虚拟的问题】：{err_list[i][1]},\n【标准对应问题】：{err_list[i][2]},\n【匹配的QA问题】：{err_list[i][3]},\n【匹配的QA问题对应的答案】：{err_list[i][4]}【Qwen判断结果为】：{err_list[i][5]}\n")

【错误类型】：TF,
【虚拟的问题】：我了解到选择个人保险时有很高的灵活性，大人和孩子能够分别定制不同的保障方案，能否详细介绍一下这种组合方式？,
【标准对应问题】：听说精选个人投保选项比较灵活，大人小孩可以拆分不同的福利搭配投保，具体可以怎样搭配？,
【匹配的QA问题】：一个家庭客户，爸爸选择欣享，妈妈和孩子选择投保精选，可以给到家庭折扣吗？或者家庭客户在精选方案内选择不同方案搭配，可以给到家庭折扣吗？,
【匹配的QA问题对应的答案】：前者不能给到家庭折扣。如果分开选择产品，则遵从各方案对于家庭折扣的规定，而不能跨产品享受家庭折扣。2024精选方案被保险人员方案即便搭配不一样也支持家庭折扣。
在以上提问的案例中，爸爸一个人选择欣享人生，则不享受家庭折扣；如果妈妈和孩子选择精选个人，即便妈妈和孩子的保障选项不一致，则妈妈和孩子的费率可以适用精选的家庭折扣。2024精选个人方案在地理涵盖范围及医院涵盖范围一致的情况下，可以接受家庭成员选择不同的住院门诊福利搭配及免赔额搭配，且家庭折扣不再要求搭配一致。
【Qwen判断结果为】：已解答

【错误类型】：FT,
【虚拟的问题】：一个人的独特优点体现在哪里？经典个人的福利津贴规定是怎样的？,
【标准对应问题】：经典个人的卖点是什么？经典个人既往症福利额度是如何界定的？,
【匹配的QA问题】：经典个人的卖点是什么？经典个人既往症福利额度是如何界定的？,
【匹配的QA问题对应的答案】：经典个人为MSH保障最为全面的高端医疗个险方案，全面保障住院、门诊（无单独年限额）、高额优享福利（即附赠的体检、疫苗、眼科及牙科共享固定额度套餐）、既往症、儿童疫苗等福利，并提供可选生育选项。市场上大部分医疗险产品通常除外既往症及其相关并发症，欣享及精选也是如此，投保经典个人方案，无论是单住院投保还是住院+门诊投保，对于通过核保审核通过的既往症，设置3个月等待期，入保首年提供每人7,500元既往症年限额，之后续保既往症额度每年以50%的幅度复利逐年递升，为客户规划了对于既往症涵盖的美好前景，请参考以下案例演示:
王先生，37岁，长居上海，患有高血压，肺部结节，甲状腺结节等疾病，之前咨询投保医疗险和重疾险时，均被要求延期或者除外承保，希望投保一款为自己的保前疾病提供保障的医疗险。
我们为王先生推荐了经典大陆及港澳台增强保障计划A:
公立+私立，不含昂

In [43]:
import dashscope
from http import HTTPStatus
dashscope.api_key="sk-632f5cf28f0a43719096801cd7c2e61a"
def call_with_messages_qa(question,q,a):
    messages = "客户的问题是：\n" + question + "\n根据此问题我找到的参考问题是：\n" + q + "\n参考问题对应的答案是：\n" + a 
    messages = [{'role': 'system', 'content': '你是一个有用的助手'},
                {'role': 'system', 'content': '请协助我判断下列我找到的参考问题是否有效的回答了客户提出的问题, 请回答已解答或不相关三个单词'},
                {'role': 'user', 'content': messages}]

    response = dashscope.Generation.call(
        dashscope.Generation.Models.qwen_turbo,
        messages=messages,
        result_format='message',  # 将返回结果格式设置为 message
    )
    content = response.output.choices[0]["message"]["content"]
    return content

In [30]:
question = "我购买了你们的经典个人计划A产品，我最近需要拔智齿，请问这项治疗是否可以给我赔付？"
similarities = similarities_QA(question,k=1)
q = similarities[0]['text']
a = QA_dict[q]
print(question,"\n",q,a)
res = call_with_messages_qa(question,q,a)
print(res)

我购买了你们的经典个人计划A产品，我最近需要拔智齿，请问这项治疗是否可以给我赔付？ 
 我购买了你们的经典个人产品，我最近想要做理疗，请问你们对于理疗次数或者其他条件有限制吗，我去哪里做理疗都可以给我赔付吗？ 对于我们客户的享用中医，理疗服务的保障，有具体以下限制，详见条款：（六）理疗及中医治疗费（治疗次数限三十次）。
具体包括下列三项类型费用：
1. 接受由具有专业中医资格的医师实施的针灸治疗、顺势疗法所发生的治疗费。
2. 接受由具有相应医疗职业资格的医师实施的物理治疗、美式脊椎矫正、职业疗法、语言治疗（包括为治疗吞咽障碍由语言治疗师实施的电子喉镜检查）所发生的治疗费
该类治疗应当满足下列全部条件：
（1）应当为被保险人医师书面治疗计划（包括短期和长期目标，并提交保险人评估）的一部分；
（2）在合理的、可预测的时间内症状会明显好转；

未解答


In [15]:
from pyserini.search.lucene import LuceneSearcher
from pyserini.search.faiss import FaissSearcher, AutoQueryEncoder
from pyserini.search.hybrid import HybridSearcher

ssearcher = LuceneSearcher('./indexes/Insurance/bm25_index/QA_index')
encoder = AutoQueryEncoder('../Model/bge-m3')
dsearcher = FaissSearcher(index_dir = "./indexes/Insurance/dense_index/QA_index",query_encoder = encoder)
hsearcher = HybridSearcher(dsearcher, ssearcher)

import json
data_list = []
with open("./RAG_data/QA_jsonl/QA_list.jsonl", 'r', encoding='utf-8') as f:
    for line in f:
        # 每一行都是一个JSON字符串，使用json.loads转换为字典
        data_list.append(json.loads(line.strip()))

In [27]:
num = 0
total = len(QA_test_Qwen)
for i in range(total):
    hits = hsearcher.search(QA_test_Qwen[i][0], k=1, alpha=0.1)
    item1 = data_list[int(hits[0].docid)]['contents']
    if QA_test_Qwen[i][1] == item1:
        num += 1
acc = num/total
print(acc)

0.27672955974842767


In [61]:
from utils_bm25 import creat_retriever
import dashscope
from http import HTTPStatus

dashscope.api_key="sk-632f5cf28f0a43719096801cd7c2e61a"
def call_with_messages_qa(question,q,a):
    messages = "客户的问题是：\n" + question + "\n根据此问题我找到的参考问题是：\n" + q + "\n参考问题对应的答案是：\n" + a 
    messages = [{'role': 'system', 'content': '你是一个有用的助手'},
                {'role': 'system', 'content': '请协助我判断下列我找到的参考问题是否有效的回答了客户提出的问题, 请回答已解答或不相关三个单词'},
                {'role': 'user', 'content': messages}]

    response = dashscope.Generation.call(
        dashscope.Generation.Models.qwen_turbo,
        messages=messages,
        result_format='message',  # 将返回结果格式设置为 message
    )
    content = response.output.choices[0]["message"]["content"]
    return content

def similarities_QA(question,k=1):
    retriever = creat_retriever("QA")
    queries=[question]
    print(queries)
    similarities = retriever.retrieve(queries=queries, k=k)
    mate_query = similarities[0]
    return mate_query

def QA_math(QA_dict, question):
    similarities = similarities_QA(question,k=1)
    q = similarities[0]['text']
    a = QA_dict[q]
    res = call_with_messages_qa(question,q,a)
    if res == "已解答":
        return 0, q, a
    elif res == "不相关":
        return 1, q, a

In [62]:
question = "我购买了你们的经典个人计划A产品，护理费是可以全额理赔的是吗？存在什么限制吗？"
ide, q, a = QA_math(QA_dict=QA_dict, question=question)
if ide == 0:
    print("根据您输入的问题,您可以参考一下问题以及回答")
    print("[参考问题]:",q)
    print("[参考回答]:",a)
if ide == 1:
    print("启动条款匹配程序")

['我购买了你们的经典个人计划A产品，护理费是可以全额理赔的是吗？存在什么限制吗？']
根据您输入的问题,您可以参考一下问题以及回答
[参考问题]: 我购买了你们的经典个人产品，护理费是可以全额理赔的是吗？存在什么限制吗？
[参考回答]: A：您好，我们对于护理费的理赔有以下条件限制，详见以下条款：（八）专业护士家庭护理费。专业护士家庭护理应 当同时满足以下全部条件： 1. 被保险人对专业护士护理确有需要，由医师推荐的、非出于方便家庭成员目的；2. 专业护士每日护理时间不超过十二小时，但下列情形不在此限：（1）住院治疗的被保险人被运送回家当日；（2）被保险人病情急重，应当接受十二小时以上专业护士护理以免入院接受治疗；（3）根据医疗实践和标准，被保险人需要在专业护理机构接受护理，但专业护理机构没有空余床位。（九）耐用医疗设备购买费或者租赁费。对于患癌症且接受属于保险责任范围的乳房切除术的被保险人，两义乳及可放入义乳的胸衣费用也属于该项费用。



In [29]:
real_num = 0
for i in range(len(q_list_test)):
    ans = test_all(q_list_test[i])
    if ans == 0:
        test_single(q_list_test[i])
    real_num += test_all(q_list_test[i])
acc = real_num/len(q_list_test)
print(f"已完成所有{len(q_list_test)}条数据的测试，正确率为：{acc}")

测试的输入问题为：
目前如果正在怀孕，可以购买你们的产品吗？

匹配的标准问题为：
目前在孕的客户是否可以购买这几款个人医疗保险产品？

匹配的回答是：
不可以。如果客户已经怀孕了，除全球孕中险（MC3）和美中宜和孕倍安保险外，经典个人、精选个人、欣享人生等保险计划均不能接受投保申请。建议客户生产完成后投保，如实告知，并附上生产出院小结供核保审核。


标准答案是：
如果客户已经怀孕了，除全球孕中险（MC3）和美中宜和孕倍安保险外，经典个人、精选个人、欣享人生等保险计划均不能接受投保申请。建议客户生产完成后投保，如实告知，并附上生产出院小结供核保审核。


错误

测试的输入问题为：
如果我投保后经典个人后发生了流产，是否可以赔付呢？

匹配的标准问题为：
MSH个人医疗保险产品中，如果购买了生育套餐可以直接给到发票理赔吗？

匹配的回答是：
个人医疗保险产品中若涵盖生育福利的（经典个人），客户购买了生育套餐后，需要在在实际生育疗程全部完成后将发票和每次产检，生产病历报告收集完整后一并递交理赔。


标准答案是：
选择性流产是保险除外责任，经典个人方案中是不赔付的。医学必要性的流产，如果是先兆流产，赔付在生育福利额度里；如果是异位妊娠终止、妊娠期内无法继续妊娠时终止妊娠或稽留流产，则属于在妊娠并发症范畴。


错误

测试的输入问题为：
你们MSH的不同产品间的核保都是什么样的？

匹配的标准问题为：
这几款个人医疗保险产品的核保条件是什么样的？

匹配的回答是：
不同个人医疗保险产品的核保条件不同，所有产品都是需要先通过核保后方可进入投保流程。以下为保险条款对于既往症的定义:
在保险人对其保险责任生效前被保险人已就此接受诊断、医学咨询或治疗，或服用药物，或显现症状的疾病或损伤。
几款产品的核保条件如下:
经典个人计划——人工核保，既往症承接、延期或者拒保；未选择可选生育福利的客户，经审核通过的既往症会有3个月等待期，等待期后第一年既往症限额为7,500元。对连续投保的客户，从第二个保险年度起，既往症及其相关并发症的保障额度按每年50%递增。未经审核的既往症将不予理赔。但请留意经典方案选择生育福利的客户，在生育福利保障期间不享有既往症福利。
精选个人计划——人工核保，既往症除外承接、延期或者拒保；MSH团体客户转精选个人的，经核保通过的既往症，可限额承担既往症责任


In [31]:
34/35

0.9714285714285714